In [3]:
import json
from typing import List
import random
from transformers import BertTokenizer, BertModel, pipeline
from scipy.spatial import distance
import os
os.environ["HF_ENDPOINT"] = "https://huggingface.co"

# Loading data

In [4]:
def read_json(file_path):
    with open(file_path, encoding='utf-8') as f:
        data = json.load(f)
    return data

In [5]:
FILE_PATH = '..\\Data\\aligned\\20170207_test_alignment.json'
data = read_json(FILE_PATH)

# Dividing the alignments into sentences

In [6]:
def alignments_to_sentences(alignments: List[List[str]]):
    all_sentences = []
    new_sent_transcript, new_sent_gold_label = [], []
    for alignment in alignments:
        new_sent_transcript.append(alignment[0])
        new_sent_gold_label.append(alignment[1])
        if alignment[0] in ['.', '!', '?']:
            all_sentences.append([new_sent_transcript, new_sent_gold_label])
            new_sent_transcript, new_sent_gold_label = [], []
    return all_sentences

In [7]:
sentences = alignments_to_sentences(alignments=data)
print(sentences[15])

[['Ketil', 'kjenseth', 'vær', 'så', 'god', '.'], ['Ketil', 'Kjenseth', 'vær', 'så', 'god', '.']]


In [44]:
with open("sentences.json", "w") as file:
    file.write(json.dumps(sentences))

In [8]:
print(len(sentences))

553


# Two approaches for SWER
## Approach 1
Function that receives BERT tokenizer & model as input and returns embeddings

Challenges:
 - Cosine distance between similar words
 - What to do with [CLS] and [SEP] tokens

## Approach 2
Function that receives huggingface feature-extractor pipeline as input

Challenges:
 - Normalize SWER(?)

In [11]:
def get_model_and_tokenizer(bert_model):
    tokenizer = BertTokenizer.from_pretrained(bert_model)
    model = BertModel.from_pretrained(bert_model)
    return model, tokenizer


def list_to_bert_embeddings(words_list, model, tokenizer):
    input_ids = tokenizer.encode(words_list, return_tensors='pt')
    output = model(input_ids)
    embeddings = output[0][0, :, :]
    return embeddings

def equalize_lists(list1, list2):
    if len(list1) < len(list2):
        list1 += [''] * (len(list2) - len(list1))
    elif len(list2) < len(list1):
        list2 += [''] * (len(list1) - len(list2))
    return list1, list2

def wer(sent1: List[str], sent2: List[str], case_sensitive=True):
    if not len(sent1) == len(sent2):
        sent1, sent2 = equalize_lists(sent1, sent2)

    errors = 0
    for w1, w2 in zip(sent1, sent2):
        if case_sensitive:
            if w1 != w2:
                errors += 1
        else:
            if w1.lower() != w2.lower():
                errors += 1
    return errors / len(sent1)

def get_n_most_severe(lst, n):
    largest_indexes = []
    largest_values = []
    for i, num in enumerate(lst):
        if len(largest_values) < n:
            largest_indexes.append(i)
            largest_values.append(num)
            largest_values, largest_indexes = (list(x) for x in zip(*sorted(zip(largest_values, largest_indexes), reverse=True)))
        elif num > largest_values[-1]:
            largest_values[-1] = num
            largest_indexes[-1] = i
            largest_values, largest_indexes = (list(x) for x in zip(*sorted(zip(largest_values, largest_indexes), reverse=True)))
    return largest_values, largest_indexes

def remove_stopwords(list1, list2, stopwords):    
    filtered_list1 = []
    filtered_list2 = []
    
    for word1, word2 in zip(list1, list2):
        if word1 not in stopwords and word2 not in stopwords:
            filtered_list1.append(word1)
            filtered_list2.append(word2)
    
    return filtered_list1, filtered_list2

def remove_punctuation(list1, list2):    
    filtered_list1 = []
    filtered_list2 = []
    punctuation = [',', '.', '!', '?']

    for word1, word2 in zip(list1, list2):
        if word1 not in punctuation and word2 not in punctuation:
            filtered_list1.append(word1)
            filtered_list2.append(word2)
    
    return filtered_list1, filtered_list2

def swer(pipe: pipeline, sent1: List[str], sent2: List[str], case_insensitive=False, 
        print_severity=False, n_severe=3, activation_function=None, stopwords=None, exclude_punctuation=False):
    
    if not len(sent1) == len(sent2):
        sent1, sent2 = equalize_lists(sent1, sent2)

    if stopwords is not None:
        sent1, sent2 = remove_stopwords(sent1, sent2, stopwords)

    if exclude_punctuation:
        sent1, sent2 = remove_punctuation(sent1, sent2)

    if case_insensitive:
        sent1 = [word.lower() for word in sent1]
        sent2 = [word.lower() for word in sent2]

    embeddings1, embeddings2 = pipe(sent1), pipe(sent2)
    distances = [distance.cosine(e1[0][1], e2[0][1]) for e1, e2 in zip(embeddings1, embeddings2)]

    if activation_function is not None:
        cosine_distance = sum([activation_function(x) for x in distances])
    else:
        cosine_distance = sum(distances)

    if print_severity:
        most_severe, most_severe_indexes = get_n_most_severe(distances, n_severe)
        for score, i in zip(most_severe, most_severe_indexes):
            print(sent1[i], sent2[i], score)

    return cosine_distance / len(sent1)

def binary_step(value, threshold=0.5):
    return 1 if value >= threshold else 0

In [60]:
# model, tokenizer = get_model_and_tokenizer(MODEL_NAMES[0])
MODEL_NAMES = ['ltg/norbert', 'ltgoslo/norbert2', 'NbAiLab/nb-bert-base']
norbert = pipeline('feature-extraction', model=MODEL_NAMES[0], tokenizer=MODEL_NAMES[0])
norbert2 = pipeline('feature-extraction', model=MODEL_NAMES[1], tokenizer=MODEL_NAMES[1])
nbbert = pipeline('feature-extraction', model=MODEL_NAMES[2], tokenizer=MODEL_NAMES[2])

Some weights of the model checkpoint at ltg/norbert were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ltgoslo/norbert2 were not used when initializing BertModel: ['cls.predictions.transform.L

# Experimenting
## Trying the different models on random sentences

In [38]:
example_sentences = sentences[19:27]

def test_swer(pipe: pipeline, sentences, stopwords=None):
    for sent in sentences:
        swer_score = swer(pipe, sent[0], sent[1])
        print('Transcript:', ' '.join(sent[0]))
        print('Gold label:', ' '.join(sent[1]))
        print('SWER:', swer_score)
        print('Binary-step SWER:', swer(pipe, sent[0], sent[1], activation_function=binary_step))
        print('Case-insensitive SWER', swer(pipe, sent[0], sent[1], case_insensitive=True))
        print('No Punctuation SWER', swer(pipe, sent[0], sent[1], exclude_punctuation=True))
        print('WER:', wer(sent[0], sent[1]))
        print('Case insensitive WER:', wer(sent[0], sent[1], case_sensitive=False))

        if stopwords:
            print('Stop Words:', swer(pipe, sent[0], sent[1], stopwords=stopwords))
            print(remove_stopwords(sent[0], sent[1], stopwords=stopwords))

        print()

In [50]:
# Test cell
test_swer(norbert, [["Det er ikke søtt og møtet hevet .".split(" "), "det er ikke sett . og møtet".split(" ")]])

Transcript: Det er ikke søtt og møtet hevet .
Gold label: det er ikke sett . og møtet 
SWER: 0.2356671467508068
Binary-step: 0.0
Case-insensitive SWER 0.19215762567107025
WER: 0.75



In [36]:
test_swer(norbert, [[[","], ["Hei"]]])

Transcript: ,
Gold label Hei
SWER: 0.5409684787051002
Binary-step: 1.0
Case-insensitive SWER 0.4894386537452724
WER: 1.0



In [13]:
for model in [norbert, norbert2, nbbert]:
    test_swer(model, example_sentences)

Transcript: Det opereres med ulike tall på forekomst av autisme , men cirka 1 % av befolkningen har autisme .
Gold label det opereres med ulike tall på forekomst av autisme  men cirka én prosent av befolkningen har autisme .
SWER: 0.06111547560772233
Binary-step: 0.0
Case-insensitive SWER 0.04279567725835959
WER: 0.21052631578947367

Transcript: Dessverre er oppfølgingen av barn , ungdom og voksne med autisme svært varierende , både med hensyn til hvilken kommune man bor i og hvilken helseregion eller hvilket helseforetak man tilhører .
Gold label dessverre er oppfølgingen av barn  ungdom og voksne med autisme svært varierende  både med hensyn til hvilken kommune man bor i og hvilken helseregion eller hvilket helseforetak man tilhører .
SWER: 0.031175405020930014
Binary-step: 0.0
Case-insensitive SWER 0.021138317744987735
WER: 0.09375

Transcript: Mangel på nasjonale faglige retningslinjer fører til at spesialisthelsetjenesten ofte gir behandling som ikke er kunnskapsbasert .
Gold labe

In [40]:
stopwords_file = open('../stopwords.txt', encoding='utf-8')
stopwords = stopwords_file.read()
stopwords = stopwords.split()

for model in [norbert, norbert2, nbbert]:
    test_swer(model, example_sentences, stopwords=stopwords)

Transcript: Det opereres med ulike tall på forekomst av autisme , men cirka 1 % av befolkningen har autisme .
Gold label: det opereres med ulike tall på forekomst av autisme  men cirka én prosent av befolkningen har autisme .
SWER: 0.06111547560772233
Binary-step: 0.0
Case-insensitive SWER 0.04279567725835959
WER: 0.21052631578947367
Stop Words: 0.06775982232573602
(['opereres', 'ulike', 'tall', 'forekomst', 'autisme', ',', 'cirka', '1', '%', 'befolkningen', 'autisme', '.'], ['opereres', 'ulike', 'tall', 'forekomst', 'autisme', '', 'cirka', 'én', 'prosent', 'befolkningen', 'autisme', '.'])

Transcript: Dessverre er oppfølgingen av barn , ungdom og voksne med autisme svært varierende , både med hensyn til hvilken kommune man bor i og hvilken helseregion eller hvilket helseforetak man tilhører .
Gold label: dessverre er oppfølgingen av barn  ungdom og voksne med autisme svært varierende  både med hensyn til hvilken kommune man bor i og hvilken helseregion eller hvilket helseforetak man t

# Testing random sentences
To compare the magnitude of SWER to WER

In [92]:
def test_n_random_sents(n: int, pipe: pipeline):
    sents = list()
    for _ in range(n):
        sent1 = random.sample(random.sample(sentences, 1)[0], 1)[0]
        sent2 = random.sample(random.sample(sentences, 1)[0], 1)[0]
        sents.append([sent1, sent2])
    test_swer(pipe, sents)

for model in [norbert, norbert2, nbbert]:
    test_n_random_sents(1, model)
    break


Transcript: Den gir en unik innsikt i det å både være foreldre .
Gold label: men problemet er volum og kapasitet .     
SWER: 0.3302967051517407
Binary-step: 0.08333333333333333
Case-insensitive SWER 0.3177094981880563
WER: 1.0



# Artificially Produced Sentences

In [111]:
sentence1 = [
    ["Det", "var", "virkelig", "en", "bra", "dag", "."],
    ["Det", "var", "ikke", "en", "bra", "dag", "."]
]

sentence2 = [
    ["Kjøretøyet", "suste", "unna", "i", "høy", "fart", "."],
    ["Bilen", "kjørte", "vekk", "i", "enorm", "hastighet", "."]
]

sentence3 = [
    ["På", "fredag", ",", "så", "dro", "jeg", "ikke", "på", "jobb", "fordi", "jeg", "var", "syk"],
    ["På", "fredag", "så", "dro", "jeg", "ikke", "på", "jobb", "fordi", "jeg", "var", "syk"]
]
artifical_sentences = [sentence1, sentence2, sentence3]

In [112]:
test_swer(norbert2, artifical_sentences)

Transcript: Det var virkelig en bra dag .
Gold label: Det var ikke en bra dag .
SWER: 0.04357761561948254
Binary-step: 0.0
Case-insensitive SWER 0.04357761561948254
WER: 0.14285714285714285

Transcript: Kjøretøyet suste unna i høy fart .
Gold label: Bilen kjørte vekk i enorm hastighet .
SWER: 0.2012825053771187
Binary-step: 0.0
Case-insensitive SWER 0.171623242925143
WER: 0.7142857142857143

Transcript: På fredag , så dro jeg ikke på jobb fordi jeg var syk
Gold label: På fredag så dro jeg ikke på jobb fordi jeg var syk 
SWER: 0.30230793441218906
Binary-step: 0.15384615384615385
Case-insensitive SWER 0.30230793441218906
WER: 0.8461538461538461



# Case sensitive
## With xlm-roberta and s-bert

In [15]:
# All sentences case sensitive
score_dict = dict()
score_dict["wer"] = [wer(sent[0], sent[1]) for sent in sentences]

model_scores = []
for model, modelname in zip([norbert, norbert2, nbbert], ["norbert", "norbert2", "nb-bert"]):
    swer_scores = [swer(model, sent[0], sent[1])  for sent in sentences]
    model_scores.append(sum(swer_scores) / len(swer_scores))
    score_dict[modelname] = swer_scores


print(model_scores)

[0.08385534497891015, 0.11024356437605232, 0.12780364525268315]


In [16]:
roberta = pipeline('feature-extraction', model="xlm-roberta-base", tokenizer="xlm-roberta-base")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
roberta_swer_scores = [swer(roberta, sent[0], sent[1])  for sent in sentences]
score_dict["roberta"] = roberta_swer_scores

In [18]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
sbert = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [19]:
hypothesis_sentences = [sent[0] for sent in sentences]
goldlabel_sentences = [sent[1] for sent in sentences]

embs1 = sbert.encode(hypothesis_sentences)
embs2 = sbert.encode(goldlabel_sentences)

embs1 = np.array(embs1).tolist()
embs2 = np.array(embs2).tolist()

cos_dists = [distance.cosine(x, y) for x, y in zip(embs1, embs2)]

score_dict["sbert"] = cos_dists

In [20]:
with open("swer_scores_case_sensitive.json", "w") as file:
    file.write(json.dumps(score_dict))

# Case insensitive

In [15]:
# On all sentences case insensitive
score_dict_case_insensitive = dict()
score_dict_case_insensitive["wer"] = [wer(sent[0], sent[1], case_sensitive=False) for sent in sentences]
model_scores_case_insensitive = []
scores_per_model = []
for model, modelname in zip([norbert, norbert2, nbbert], ["norbert", "norbert2", "nb-bert"]):
    swer_scores = [swer(model, sent[0], sent[1], case_insensitive=True)  for sent in sentences]
    scores_per_model.append(swer_scores)
    model_scores_case_insensitive.append(sum(swer_scores) / len(swer_scores))
    score_dict_case_insensitive[modelname] = swer_scores

print(model_scores_case_insensitive)

[0.056021692850268125, 0.08637109727186182, 0.10615662187031918]


In [17]:
with open("swer_scores_case_insensitive.json", "w") as file:
    file.write(json.dumps(score_dict_case_insensitive))

# Activation function

In [27]:
score_dict_binary_step = dict()
score_dict_binary_step["wer"] = [wer(sent[0], sent[1]) for sent in sentences]
model_scores = []
for model, modelname in zip([norbert, norbert2, nbbert], ["norbert", "norbert2", "nb-bert"]):
    swer_scores = [swer(model, sent[0], sent[1], activation_function=binary_step)  for sent in sentences]
    model_scores.append(sum(swer_scores) / len(swer_scores))
    score_dict_binary_step[modelname] = swer_scores

print(model_scores)

[0.035849470664348924, 0.12131628258153258, 0.12133948754994434]


In [29]:
with open("swer_scores_binarystep.json", "w") as file:
    file.write(json.dumps(score_dict_binary_step))

# Without punctuation

In [30]:
model_scores = []
score_dict_no_punctuation = dict()
score_dict_no_punctuation["wer"] = [wer(sent[0], sent[1]) for sent in sentences]
for model, modelname in zip([norbert, norbert2, nbbert], ["norbert", "norbert2", "nb-bert"]):
    swer_scores = [swer(model, sent[0], sent[1], exclude_punctuation=True)  for sent in sentences]
    model_scores.append(sum(swer_scores) / len(swer_scores))
    score_dict_no_punctuation[modelname] = swer_scores

print(model_scores)

[0.07304126945090592, 0.07680017171337927, 0.09286934846219716]


In [31]:
with open("swer_scores_no_punctuation.json", "w") as file:
    file.write(json.dumps(score_dict_no_punctuation))

# With Stop Words

In [33]:
stopwords_file = open('../stopwords.txt', encoding='utf-8')
stopwords = stopwords_file.read()
stopwords = stopwords.split()

In [34]:
model_scores = []
score_dict_stopwords = dict()
score_dict_stopwords["wer"] = [wer(sent[0], sent[1]) for sent in sentences]
for model, modelname in zip([norbert, norbert2, nbbert], ["norbert", "norbert2", "nb-bert"]):
    swer_scores = [swer(model, sent[0], sent[1], stopwords=stopwords)  for sent in sentences]
    model_scores.append(sum(swer_scores) / len(swer_scores))
    score_dict_stopwords[modelname] = swer_scores

print(model_scores)

[0.09293402972352456, 0.13646640414266845, 0.1595510255410295]


In [35]:
with open("swer_scores_stopwords.json", "w") as file:
    file.write(json.dumps(score_dict_stopwords))

# Grouping by severity

In [19]:
swer_scores_norbert2_case_insensitive = score_dict_case_insensitive["norbert2"]

In [21]:
# Grouping by severity on sentence level
def group_by_severity(sentences, scores, low_threshold=0.15, high_threshold=0.3):
    low, medium, high = [], [], []
    # for score_list in scores_per_model:
    for sentence, score in zip(sentences, scores):
        if score <= low_threshold:
            low.append((sentence, score))
        elif score > low_threshold and score <= high_threshold:
            medium.append((sentence, score))
        else:
            high.append((sentence, score))
    return low, medium, high
low, medium, high = group_by_severity(sentences, swer_scores_norbert2_case_insensitive)
print(len(low), len(medium), len(high))

482 48 23


In [31]:
for sent in low:
    print("Setning 1:", " ".join(sent[0][0]))
    print("Setning 2:", " ".join(sent[0][1]))
    print()


Setning 1: President på vegne av regjeringen har    nære overbringe 3 kongelige proposisjoner  oppheving av lov om ferie for fiskere og endringer i ferieloven .
Setning 2: president på vegne av regjeringen har jeg den ære å overbringe tre kongelige proposisjoner . opphevinga av lov om ferie for fiskere og endringer i ferieloven .

Setning 1: Endring i markedsføringsloven .
Setning 2: endring i markedsføringsloven 

Setning 1: Unntak for rett  inn til innsyn i dagligvareportalens prisopplysninger  og samtykke til inngåelse av avtale mellom Norge og Finland om fisket i tanavassdraget av 30 .
Setning 2: unntak for rett til in til innsyn i dagligvareportalens prisopplysninger . og samtykke til inngåelse av avtale mellom Norge og Finland om fiske i tanavassdraget av  tredevte

Setning 1: September totusenogseksten .
Setning 2: september totusenogseksten .

Setning 1: Proposisjonene vil bli behandlet på reglementsmessig måte .
Setning 2: proposisjonene vil bli behandlet på reglementsmessig m

In [25]:
# With severity and case sensitive
for model in [norbert, norbert2, nbbert]:
    for sent in example_sentences:
        print(" ".join(sent[0]))
        print(" ".join(sent[1]))
        swer(model, sent[0], sent[1], print_severity=True)
        print()
    # Break to only test 1 model
    break

Det opereres med ulike tall på forekomst av autisme , men cirka 1 % av befolkningen har autisme .
det opereres med ulike tall på forekomst av autisme  men cirka én prosent av befolkningen har autisme .
Det det 0.34807616863789215
,  0.33821308391980376
1 én 0.28821634862513

Dessverre er oppfølgingen av barn , ungdom og voksne med autisme svært varierende , både med hensyn til hvilken kommune man bor i og hvilken helseregion eller hvilket helseforetak man tilhører .
dessverre er oppfølgingen av barn  ungdom og voksne med autisme svært varierende  både med hensyn til hvilken kommune man bor i og hvilken helseregion eller hvilket helseforetak man tilhører .
,  0.33821308391980376
,  0.33821308391980376
Dessverre dessverre 0.32118679283015295

Mangel på nasjonale faglige retningslinjer fører til at spesialisthelsetjenesten ofte gir behandling som ikke er kunnskapsbasert .
mangel på nasjonale faglige retningslinjer fører til at spesialisthelsetjenesten ofte gir behandling som ikke er kunns

In [26]:
# With severity and case insensitive
for model in [norbert, norbert2, nbbert]:
    for sent in example_sentences:
        print(" ".join(sent[0]))
        print(" ".join(sent[1]))
        swer(model, sent[0], sent[1], print_severity=True, case_insensitive=True)
        print()
    # Break to only test 1 model
    break

Det opereres med ulike tall på forekomst av autisme , men cirka 1 % av befolkningen har autisme .
det opereres med ulike tall på forekomst av autisme  men cirka én prosent av befolkningen har autisme .
,  0.33821308391980376
1 én 0.28821634862513
% prosent 0.18668843536389845

Dessverre er oppfølgingen av barn , ungdom og voksne med autisme svært varierende , både med hensyn til hvilken kommune man bor i og hvilken helseregion eller hvilket helseforetak man tilhører .
dessverre er oppfølgingen av barn  ungdom og voksne med autisme svært varierende  både med hensyn til hvilken kommune man bor i og hvilken helseregion eller hvilket helseforetak man tilhører .
,  0.33821308391980376
,  0.33821308391980376
oppfølgingen oppfølgingen 0

Mangel på nasjonale faglige retningslinjer fører til at spesialisthelsetjenesten ofte gir behandling som ikke er kunnskapsbasert .
mangel på nasjonale faglige retningslinjer fører til at spesialisthelsetjenesten ofte gir behandling som ikke er kunnskapsbasert

# Combination of best params


## Lowercase, no punctuation

In [22]:
model_scores = []
score_dict_case_insensitive_no_punctuation = dict()
for model, modelname in zip([norbert, norbert2, nbbert], ["norbert", "norbert2", "nb-bert"]):
    swer_scores = [swer(model, sent[0], sent[1], exclude_punctuation=True, case_insensitive=True)  for sent in sentences]
    model_scores.append(sum(swer_scores) / len(swer_scores))
    score_dict_case_insensitive_no_punctuation[modelname] = swer_scores

print(model_scores)


[0.03954934965167156, 0.04941229602250747, 0.06762604140348276]


In [23]:
with open("swer_scores_case_insensitive_no_punctuation.json", "w") as file:
    file.write(json.dumps(score_dict_case_insensitive_no_punctuation))

## Lowercase, no punctuation, binary step

In [24]:
model_scores = []
score_dict_case_insensitive_no_punctuation_binarystep = dict()
for model, modelname in zip([norbert, norbert2, nbbert], ["norbert", "norbert2", "nb-bert"]):
    swer_scores = [swer(model, sent[0], sent[1], exclude_punctuation=True, case_insensitive=True, activation_function=binary_step)  for sent in sentences]
    model_scores.append(sum(swer_scores) / len(swer_scores))
    score_dict_case_insensitive_no_punctuation_binarystep[modelname] = swer_scores

print(model_scores)

[0.024049372593268746, 0.06076184206717672, 0.06463964316493882]


In [25]:
with open("swer_scores_case_insensitive_no_punctuation_binarystep.json", "w") as file:
    file.write(json.dumps(score_dict_case_insensitive_no_punctuation_binarystep))


# Computing SWER for Survey Sentences

In [48]:
with open("hwer_results.json", "r", encoding='utf-8') as file:
    hwer_sentences = json.load(file)
    hwer_sentences_swer_scores = dict()
    for model, modelname in zip([norbert, norbert2, nbbert], ["norbert", "norbert2", "nb-bert"]):
        hwer_sentences_swer_scores[modelname] = [swer(model, sent["transcript"].split(" "), sent["gold_label"].split(" ")) for sent in hwer_sentences]

In [49]:
with open("survey_sentences_swer.json", "w") as file:
    file.write(json.dumps(hwer_sentences_swer_scores))

# Printing severity 

In [74]:
with open("sentences_grouped.json", "r") as file:
    groups = json.load(file)

comb2_norbert1_high = groups['swer_scores_case_insensitive_no_punctuation_binarystep']["norbert"][2]
for sentence in comb2_norbert1_high:
    print(sentence[0])
    swer(norbert, sentence[0][0], sentence[0][1], case_insensitive=True, exclude_punctuation=True, activation_function=binary_step, print_severity=True)

[['Man', 'otte', 'og', '20', 'år', '.'], ['', '', 'mann', 'åtteogtyve', 'år', '.']]
20 åtteogtyve 0.6927403485026142
otte  0.6656763824923477
man  0.3763919955990621
[['Gutt', ',', '17', 'år', '.'], ['', 'gutt', 'sytten', 'år', '.']]
17 sytten 0.6348644083416536
gutt  0.3693435389620866
år år 0
[['Anette', 'drangsholt', ',', 'som', 'er', 'leder', 'i', 'autismeforen', '.'], ['', 'Annette', 'Drangsholt', 'som', 'er', 'leder', 'i', '', 'Autismeforen']]
drangsholt annette 0.7763517248772179
autismeforen  0.6639741851321754
anette  0.6240823615051994
[['Utvalget', 'foreslår', '8', '.'], ['utvalet', 'foreslår', '', '']]
utvalget utvalet 0.6800571133678861
8  0.47606610192278076
foreslår foreslår 0
[['Anders', '.'], ['', 'Andersen']]
anders  0.6831326258039402
[['Ja', '.'], ['', 'hevet']]
ja  0.5107651298654483


In [81]:
default_norbert_high = groups['swer_scores_case_sensitive']["norbert"][2]
for sentence in default_norbert_high:
    print(sentence[0])
    swer(norbert, sentence[0][0], sentence[0][1], print_severity=True)

[['Hei', '.'], ['', '']]
Hei  0.45558837740800295
.  0.30760792749229904
[['Det', 'er', 'ikke', 'det', '.'], ['', '', '', '', '']]
det  0.473436679497107
Det  0.4675967118911589
er  0.39828284858651664
[['Hei', '.'], ['', '']]
Hei  0.45558837740800295
.  0.30760792749229904
[['Hei', '.'], ['', '']]
Hei  0.45558837740800295
.  0.30760792749229904
[['Hei', '.'], ['', '']]
Hei  0.45558837740800295
.  0.30760792749229904
[['Hei', '.'], ['', '']]
Hei  0.45558837740800295
.  0.30760792749229904
[['Hei', '.'], ['', '']]
Hei  0.45558837740800295
.  0.30760792749229904
[['Representanten', 'freddy', 'de', 'ruiter', '.'], ['representanten', 'Freddy', 'de', '', 'Ruiter']]
. Ruiter 0.6237585613645859
ruiter  0.5922591599902027
freddy Freddy 0.5486587810775494
[['Man', 'otte', 'og', '20', 'år', '.'], ['', '', 'mann', 'åtteogtyve', 'år', '.']]
20 åtteogtyve 0.6927403485026142
otte  0.6656763824923477
Man  0.42948881691686025
[['Gutt', ',', '17', 'år', '.'], ['', 'gutt', 'sytten', 'år', '.']]
17 sytte

In [75]:
default_norbert_medium = groups['swer_scores_case_sensitive']["norbert"][1]
i = 0
for sentence in default_norbert_medium:
    if i == 10:
        break
    print(sentence[0])
    swer(norbert, sentence[0][0], sentence[0][1], print_severity=True)
    i += 1

[['Stortingets', 'møte', 'er', 'lovlig', ',', 'satt', '', 'representantene', 'Fredrik', 'helen', 'fredric', 'holen', 'Bjørdal', 'og', 'Trond', 'Giske', ',', 'som', 'har', 'vært', 'permitterte', ',', 'har', 'igjen', 'tatt', 'sete', '', 'følgende', 'vararepresentanter', 'tar', 'nå', 'sete', '', '', 'nei', 'for', 'Vest', '', 'Agder', 'fylke', ',', 'anen', 'Verdal', 'for', 'Akershus', 'fylke', ',', 'Kjartan', 'berland', 'og', '', 'are', 'helseth', 'for', 'Buskerud', 'fylke', ',', 'lavrans', 'kierulf', 'for', 'Oslo', ',', 'ann', 'Kathrine', '', 'skjørshammer', 'for', '.'], ['Stortingets', 'møte', 'er', 'lovlig', '', 'satt', '.', 'representantene', 'Fredric', 'Helen', 'Fredric', 'Holen', 'Bjørdal', 'og', 'Trond', 'Giske', '', 'som', 'har', 'vært', 'permitterte', '', 'har', 'igjen', 'tatt', 'sete', '.', 'følgende', 'vararepresentanter', 'tar', 'nå', 'sete', '.', 'det', 'er', 'for', 'Vest', '-', 'Agder', 'fylke', 'Ånen', 'Werdal', '.', 'for', 'Akershus', 'fylke', '', 'Kjartan', 'Berland', 'og'

In [82]:
default_norbert_low = groups['swer_scores_case_sensitive']["norbert"][0]
for sentence in default_norbert_low:
    print(sentence[0])
    swer(norbert, sentence[0][0], sentence[0][1], print_severity=True)
    i += 1

[['President', 'på', 'vegne', 'av', 'regjeringen', 'har', '', '', '', 'nære', 'overbringe', '3', 'kongelige', 'proposisjoner', '', 'oppheving', 'av', 'lov', 'om', 'ferie', 'for', 'fiskere', 'og', 'endringer', 'i', 'ferieloven', '.'], ['president', 'på', 'vegne', 'av', 'regjeringen', 'har', 'jeg', 'den', 'ære', 'å', 'overbringe', 'tre', 'kongelige', 'proposisjoner', '.', 'opphevinga', 'av', 'lov', 'om', 'ferie', 'for', 'fiskere', 'og', 'endringer', 'i', 'ferieloven', '.']]
 jeg 0.4412392418514939
nære å 0.4107474011194604
 den 0.3793114789646982
[['Unntak', 'for', 'rett', '', 'inn', 'til', 'innsyn', 'i', 'dagligvareportalens', 'prisopplysninger', '', 'og', 'samtykke', 'til', 'inngåelse', 'av', 'avtale', 'mellom', 'Norge', 'og', 'Finland', 'om', 'fisket', 'i', 'tanavassdraget', 'av', '30', '.'], ['unntak', 'for', 'rett', 'til', 'in', 'til', 'innsyn', 'i', 'dagligvareportalens', 'prisopplysninger', '.', 'og', 'samtykke', 'til', 'inngåelse', 'av', 'avtale', 'mellom', 'Norge', 'og', 'Finlan